In [1]:
# Import necessary libraries
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import evaluate 

In [2]:
# Load a pre-trained model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)  # Example: binary classification
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)  # Use fast tokenizer for speed


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Load a dataset (use your dataset if necessary)
dataset = load_dataset("imdb")  # You can change this to any dataset you are working with

# Use only a subset of the dataset for faster training (e.g., 1000 examples for training, 200 for testing)
train_data = dataset['train'].select([i for i in range(1000)])  
test_data = dataset['test'].select([i for i in range(200)])

In [7]:
# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)


In [9]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [11]:
# Set up an instance of TrainingArguments
training_args = TrainingArguments(
  output_dir="./finetuned",  # Directory to save the model and logs
  eval_strategy="epoch",  # Evaluate at the end of each epoch
  num_train_epochs=1,  # Reduce the number of epochs for faster training
  learning_rate=2e-5,  # Learning rate
  per_device_train_batch_size=3,  # Batch size for training
  per_device_eval_batch_size=3,  # Batch size for evaluation
  weight_decay=0.01,  # Weight decay for regularization
  fp16=True,  # Enable mixed precision training
  gradient_accumulation_steps=8,  # Simulate larger batch size
)

In [13]:
# Set up the Trainer class
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,  # Ensures tokenization consistency
)

C:\Users\VICTUS\AppData\Local\Temp\ipykernel_29872\311803706.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
# Load the metrics from Hugging Face's Evaluate library
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

In [17]:
# Function to compute metrics
def compute_metrics(pred):
    predictions, labels = pred
    predictions = torch.argmax(torch.tensor(predictions), dim=-1)
    
    # Compute and return the metrics
    accuracy_result = accuracy.compute(predictions=predictions, references=labels)
    precision_result = precision.compute(predictions=predictions, references=labels)
    recall_result = recall.compute(predictions=predictions, references=labels)
    f1_result = f1.compute(predictions=predictions, references=labels)

    return {
        "accuracy": accuracy_result["accuracy"],
        "precision": precision_result["precision"],
        "recall": recall_result["recall"],
        "f1": f1_result["f1"]
    }

In [19]:
# Update the Trainer to compute metrics during evaluation
trainer.compute_metrics = compute_metrics

In [21]:
# Train the model
trainer.train()

C:\Users\VICTUS\anaconda36\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['label']

In [ ]:
# Install necessary libraries (run this in your environment if not installed)
# pip install transformers datasets torch evaluate

# Import necessary libraries
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import evaluate  # Hugging Face's Evaluate library

# Load a pre-trained model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)  # Example: binary classification
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)  # Use fast tokenizer for speed

# Load a dataset (use your dataset if necessary)
dataset = load_dataset("imdb")  # You can change this to any dataset you are working with

# Use only a subset of the dataset for faster training (e.g., 1000 examples for training, 200 for testing)
train_data = dataset['train'].select([i for i in range(1000)])  # Use a smaller subset for faster training
test_data = dataset['test'].select([i for i in range(200)])

# Tokenize the data
def tokenize_function(examples):
    # Ensure that the text is tokenized and return necessary fields (input_ids, attention_mask)
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

# Apply tokenization and ensure the required fields are returned
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Split the dataset into training and testing data
train_data = tokenized_datasets['train']
test_data = tokenized_datasets['test']

# Set up an instance of TrainingArguments
training_args = TrainingArguments(
  output_dir="./finetuned",  # Directory to save the model and logs
  eval_strategy="epoch",  # Changed from 'evaluation_strategy' to 'eval_strategy'
  num_train_epochs=1,  # Reduce the number of epochs for faster training
  learning_rate=2e-5,  # Learning rate
  per_device_train_batch_size=3,  # Batch size for training
  per_device_eval_batch_size=3,  # Batch size for evaluation
  weight_decay=0.01,  # Weight decay for regularization
  fp16=True,  # Enable mixed precision training
  gradient_accumulation_steps=8,  # Simulate larger batch size
)

# Set up the Trainer class
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,  # Ensures tokenization consistency
)

# Load the metrics from Hugging Face's Evaluate library
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

# Function to compute metrics
def compute_metrics(pred):
    predictions, labels = pred
    predictions = torch.argmax(torch.tensor(predictions), dim=-1)
    
    # Compute and return the metrics
    accuracy_result = accuracy.compute(predictions=predictions, references=labels)
    precision_result = precision.compute(predictions=predictions, references=labels)
    recall_result = recall.compute(predictions=predictions, references=labels)
    f1_result = f1.compute(predictions=predictions, references=labels)

    return {
        "accuracy": accuracy_result["accuracy"],
        "precision": precision_result["precision"],
        "recall": recall_result["recall"],
        "f1": f1_result["f1"]
    }

# Update the Trainer to compute metrics during evaluation
trainer.compute_metrics = compute_metrics

# Train the model
trainer.train()

# Evaluate the model
evaluation_results = trainer.evaluate()

# Print the evaluation results
print("Evaluation Results:", evaluation_results)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

C:\Users\VICTUS\AppData\Local\Temp\ipykernel_35092\1685799575.py:47: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
